# Generate a TLE breakup

- use the EVOLVE model and break up a TLE (for now, at epoch)

In [13]:
from sgp4.io import twoline2rv
from sgp4.propagation import sgp4 as sgprop
from sgp4.earth_gravity import wgs72

from EVOLVE import evolve_collision

import numpy as np
import pandas as pd

In [59]:
TLE = '1 32258U 07046A   25308.81572472 -.00000255  00000-0  00000-0 0  9994', '2 32258   0.1483  87.0630 0001016 141.6959  66.7150  1.00271503 50082'

tleo  = twoline2rv( *TLE, wgs72 )
tlejd = tleo.jdsatepoch
epoch = pd.to_datetime( tlejd, unit='D', origin='julian')
P,V   = [np.array(X) for X in sgprop( tleo, 0)]

WGS_mass    = 5987 # kg, from Gunther
WGS_charlen = 15   # meters; guess

print(epoch)
print('\n'.join(TLE))

2025-11-04 19:34:38.615789112
1 32258U 07046A   25308.81572472 -.00000255  00000-0  00000-0 0  9994
2 32258   0.1483  87.0630 0001016 141.6959  66.7150  1.00271503 50082


In [55]:
# model this as a non-catatrophic collision
non_catastrophic = evolve_collision( WGS_mass, 25, 
                     V, 0.95*V,
                     WGS_charlen,0.1,
                     min_characteristic_length=0.01 )
non_catastrophic.match_mass()
non_catastrophic.catastrophic

Removed 0 objects to enforce mass limit (collision_mass:    3.844 oldmass: 4728.561 / newmass: 4728.561)


False

In [63]:
epochstr = epoch.strftime('%Y %m %d %H %M %S.%f')

print('''DEFINE CLOCK
        EPOCH  {}
        TIME_STEP 60\n'''.format( epoch.strftime('%Y %m %d %H %M %S') ) )

for i,R in non_catastrophic.sim_output().iterrows():
    print( 'DEFINE PLATFORM KEPLER  "{:006d}"'.format(i) )
    delV = np.hstack( (R['vel_x'],R['vel_y'],R['vel_z'] ) ) / 1e3
    newV = delV + V
    print('\tSTATE "Earth"   CARTESIAN       CUSTOM {} {} {} {} {} {} {}'.format( *P, *newV , epochstr) )
    print('\tSHAPE DOT')


Removed 0 objects to enforce mass limit (collision_mass:    3.844 oldmass: 4728.561 / newmass: 4728.561)


DEFINE CLOCK
        EPOCH  2025 11 04 19 34 38
        TIME_STEP 60

DEFINE PLATFORM KEPLER  "000000"
	STATE "Earth"   CARTESIAN       CUSTOM 18115.210595361084 -38072.60351852945 -39.912896581359455 2.781014292371852 1.3268672598966613 0.003191044355105926 2025 11 04 19 34 38.615789
	SHAPE DOT
DEFINE PLATFORM KEPLER  "000001"
	STATE "Earth"   CARTESIAN       CUSTOM 18115.210595361084 -38072.60351852945 -39.912896581359455 2.7781696160816085 1.3210044958893497 -0.0029536751809726915 2025 11 04 19 34 38.615789
	SHAPE DOT
DEFINE PLATFORM KEPLER  "000002"
	STATE "Earth"   CARTESIAN       CUSTOM 18115.210595361084 -38072.60351852945 -39.912896581359455 2.7786486197633122 1.32546806956871 0.0017249805477233477 2025 11 04 19 34 38.615789
	SHAPE DOT
DEFINE PLATFORM KEPLER  "000003"
	STATE "Earth"   CARTESIAN       CUSTOM 18115.210595361084 -38072.60351852945 -39.912896581359455 2.8521524928923374 1.3859337285049604 0.02638679350480278 2025 11 04 19 34 38.615789
	SHAPE DOT
DEFINE PLATFORM KEP

In [58]:
non_catastrophic.sim_output()

Removed 0 objects to enforce mass limit (collision_mass:    3.844 oldmass: 4728.561 / newmass: 4728.561)


,mass,A/M,area,vel_x,vel_y,vel_z
particle,,,,,,
0,0.012324,0.344465,0.004245,3.453185,1.518486,11.143659
1,0.000067,0.994219,0.000067,0.333275,2.378401,2.324410
2,0.000102,0.895777,0.000091,8.962392,1.860752,0.445369
3,0.000124,0.810953,0.000100,62.741804,83.198150,10.904063
4,1.615598,0.100030,0.161608,1.932299,9.205361,9.592191
...,...,...,...,...,...,...
717,16.106239,0.079409,1.278976,10.310039,1.423581,7.060668
718,0.000056,2.423803,0.000135,2.611440,3.969754,3.021913
719,0.000063,1.840097,0.000116,383.001094,104.990156,217.703649
